In [14]:
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import seaborn as sns
# To Plot matplotlib figures inline on the notebook
%matplotlib inline


from sklearn.cross_validation import train_test_split
from sklearn.metrics import r2_score
from sklearn import linear_model,ensemble, tree, model_selection
from sklearn.linear_model import Lasso, Ridge, LassoCV, RidgeCV

from sklearn import pipeline, manifold, preprocessing, feature_selection, decomposition



#This is for displaying the tree (we shouldn't need these now)
#!pip install pydotplus
#!pip install GraphViz
#import pydotplus as pydot
#from IPython.display import Image
from sklearn.externals.six import StringIO

# various options in pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

In [15]:
datafile = 'ModelData1.csv'

df = pd.read_csv(datafile)
df

,Unnamed: 0,ID_num,applicationCategory,OS_windows,OS_mac,OS_linux,SteamPlay,Release_Date,worstRating,bestRating,ratingValue,reviewCount,Price,Lowest_Price,Max_Sale,controller_support,metacritic_score,community_visible_stats,workshop_visible,releasestate,languages_num,community_hub_visible,owners,owners_unc,players_total,owners_played_percent,players_2_weeks,players_2_weeks_percent,median_total_playtime,average_total_playtime,Packages,DLCs,Depots,genre_Action,genre_Indie,genre_Strategy,genre_Simulation,genre_RPG,genre_Adventure,genre_Casual,genre_Early Access,genre_Racing,genre_Sports,genre_Free to Play,genre_Massively Multiplayer
0,0,10,0,1,1,1,1,2000-11-01,0.0,100.0,97.74,93913.0,9.99,2.49,75.0,0,88.0,0,-1,-1,0,0,1.344e+07,96036.0,9.427e+06,70.13,361920.0,2.69,6.9,185.9,65,0,24.0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,1002,0,-1,-1,-1,0,2005-10-12,NaN,NaN,NaN,NaN,9.99,2.49,75.0,0,69.0,-1,-1,-1,0,0,3.935e+04,5282.0,1.188e+04,30.19,0.0,0.00,NaN,1.1,4,1,2.0,0,1,0,0,0,0,0,0,0,0,0,0
2,2,10090,0,-1,-1,-1,0,2008-11-18,0.0,100.0,92.69,16971.0,19.99,9.79,51.0,0,83.0,0,-1,-1,0,0,1.674e+06,34382.0,1.424e+06,85.07,106163.0,6.34,12.5,42.1,41,0,7.0,1,0,0,0,0,0,0,0,0,0,0,0
3,3,10130,0,-1,-1,-1,0,2007-10-30,0.0,100.0,76.69,653.0,19.99,4.99,75.0,0,71.0,0,-1,-1,0,0,1.340e+05,9746.0,5.549e+04,41.41,1172.0,0.87,1.1,3.1,6,0,1.0,1,0,0,0,0,0,0,0,0,0,0,0
4,4,10180,0,-1,-1,-1,0,2009-11-12,0.0,100.0,90.93,26992.0,19.99,9.79,51.0,0,86.0,0,-1,-1,8,0,5.246e+06,60607.0,4.687e+06,89.33,70157.0,1.34,9.4,23.5,37,0,18.0,1,0,0,0,0,0,0,0,0,0,0,0
5,5,10220,0,-1,-1,-1,0,2011-12-21,NaN,NaN,NaN,NaN,11.99,4.79,60.0,0,24.0,0,-1,-1,2,0,7.830e+04,8137.0,6.901e+04,88.14,2491.0,3.18,2.1,7.1,3,0,4.0,1,0,0,0,0,0,0,0,0,0,0,0
6,6,102400,0,1,1,0,1,2011-06-10,NaN,NaN,NaN,NaN,9.99,4.99,50.0,0,70.0,0,-1,-1,0,0,3.304e+04,4840.0,1.299e+04,39.33,0.0,0.00,NaN,NaN,5,1,2.0,1,1,1,0,0,0,0,0,0,0,0,0
7,7,10250,0,-1,-1,-1,0,2009-01-30,NaN,NaN,NaN,NaN,6.99,2.49,75.0,0,NaN,-1,-1,-1,0,0,1.151e+04,2856.0,9.837e+03,85.49,0.0,0.00,1.4,2.7,4,0,3.0,0,0,0,1,0,0,0,0,0,0,0,0
8,8,102500,0,-1,-1,-1,0,2012-02-07,0.0,100.0,87.28,7748.0,19.99,4.39,78.0,3,81.0,0,-1,-1,5,0,8.919e+05,27437.0,7.470e+05,83.76,19465.0,2.18,9.2,26.6,6,4,13.0,1,0,0,0,1,0,0,0,0,0,0,0
9,9,10260,0,-1,-1,-1,0,2010-12-10,NaN,NaN,NaN,NaN,6.99,2.49,75.0,0,NaN,-1,-1,-1,0,0,1.132e+04,2833.0,6.867e+03,60.65,0.0,0.00,NaN,2.0,4,0,3.0,0,0,0,1,0,0,0,0,0,0,0,0


In [16]:
#convert Release_Date column to a datetime object
from dateutil.parser import *

def toDT(d):
    #print(d)
    if d == np.nan or d == 'nan': return np.nan
    try:
        date = parse(d)
        return date.timestamp()
    except (ValueError,TypeError):
        print("unparsable data:",d)
        return np.nan

df['Release_Date'] = df['Release_Date'].map(toDT)

unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable data: nan
unparsable da

In [17]:
#filter filter out invalid games, and set all NaN to -1

df = df.fillna(value=-1)
Adf = df.loc[df['Price'] >= 0,:]
df

,Unnamed: 0,ID_num,applicationCategory,OS_windows,OS_mac,OS_linux,SteamPlay,Release_Date,worstRating,bestRating,ratingValue,reviewCount,Price,Lowest_Price,Max_Sale,controller_support,metacritic_score,community_visible_stats,workshop_visible,releasestate,languages_num,community_hub_visible,owners,owners_unc,players_total,owners_played_percent,players_2_weeks,players_2_weeks_percent,median_total_playtime,average_total_playtime,Packages,DLCs,Depots,genre_Action,genre_Indie,genre_Strategy,genre_Simulation,genre_RPG,genre_Adventure,genre_Casual,genre_Early Access,genre_Racing,genre_Sports,genre_Free to Play,genre_Massively Multiplayer
0,0,10,0,1,1,1,1,9.731e+08,0.0,100.0,97.74,93913.0,9.99,2.49,75.0,0,88.0,0,-1,-1,0,0,1.344e+07,96036.0,9.427e+06,70.13,361920.0,2.69,6.9,185.9,65,0,24.0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,1002,0,-1,-1,-1,0,1.129e+09,-1.0,-1.0,-1.00,-1.0,9.99,2.49,75.0,0,69.0,-1,-1,-1,0,0,3.935e+04,5282.0,1.188e+04,30.19,0.0,0.00,-1.0,1.1,4,1,2.0,0,1,0,0,0,0,0,0,0,0,0,0
2,2,10090,0,-1,-1,-1,0,1.227e+09,0.0,100.0,92.69,16971.0,19.99,9.79,51.0,0,83.0,0,-1,-1,0,0,1.674e+06,34382.0,1.424e+06,85.07,106163.0,6.34,12.5,42.1,41,0,7.0,1,0,0,0,0,0,0,0,0,0,0,0
3,3,10130,0,-1,-1,-1,0,1.194e+09,0.0,100.0,76.69,653.0,19.99,4.99,75.0,0,71.0,0,-1,-1,0,0,1.340e+05,9746.0,5.549e+04,41.41,1172.0,0.87,1.1,3.1,6,0,1.0,1,0,0,0,0,0,0,0,0,0,0,0
4,4,10180,0,-1,-1,-1,0,1.258e+09,0.0,100.0,90.93,26992.0,19.99,9.79,51.0,0,86.0,0,-1,-1,8,0,5.246e+06,60607.0,4.687e+06,89.33,70157.0,1.34,9.4,23.5,37,0,18.0,1,0,0,0,0,0,0,0,0,0,0,0
5,5,10220,0,-1,-1,-1,0,1.324e+09,-1.0,-1.0,-1.00,-1.0,11.99,4.79,60.0,0,24.0,0,-1,-1,2,0,7.830e+04,8137.0,6.901e+04,88.14,2491.0,3.18,2.1,7.1,3,0,4.0,1,0,0,0,0,0,0,0,0,0,0,0
6,6,102400,0,1,1,0,1,1.308e+09,-1.0,-1.0,-1.00,-1.0,9.99,4.99,50.0,0,70.0,0,-1,-1,0,0,3.304e+04,4840.0,1.299e+04,39.33,0.0,0.00,-1.0,-1.0,5,1,2.0,1,1,1,0,0,0,0,0,0,0,0,0
7,7,10250,0,-1,-1,-1,0,1.233e+09,-1.0,-1.0,-1.00,-1.0,6.99,2.49,75.0,0,-1.0,-1,-1,-1,0,0,1.151e+04,2856.0,9.837e+03,85.49,0.0,0.00,1.4,2.7,4,0,3.0,0,0,0,1,0,0,0,0,0,0,0,0
8,8,102500,0,-1,-1,-1,0,1.329e+09,0.0,100.0,87.28,7748.0,19.99,4.39,78.0,3,81.0,0,-1,-1,5,0,8.919e+05,27437.0,7.470e+05,83.76,19465.0,2.18,9.2,26.6,6,4,13.0,1,0,0,0,1,0,0,0,0,0,0,0
9,9,10260,0,-1,-1,-1,0,1.292e+09,-1.0,-1.0,-1.00,-1.0,6.99,2.49,75.0,0,-1.0,-1,-1,-1,0,0,1.132e+04,2833.0,6.867e+03,60.65,0.0,0.00,-1.0,2.0,4,0,3.0,0,0,0,1,0,0,0,0,0,0,0,0


In [21]:
df.describe()

,Unnamed: 0,ID_num,applicationCategory,OS_windows,OS_mac,OS_linux,SteamPlay,Release_Date,worstRating,bestRating,ratingValue,reviewCount,Price,Lowest_Price,Max_Sale,controller_support,metacritic_score,community_visible_stats,workshop_visible,releasestate,languages_num,community_hub_visible,owners,owners_unc,players_total,owners_played_percent,players_2_weeks,players_2_weeks_percent,median_total_playtime,average_total_playtime,Packages,DLCs,Depots,genre_Action,genre_Indie,genre_Strategy,genre_Simulation,genre_RPG,genre_Adventure,genre_Casual,genre_Early Access,genre_Racing,genre_Sports,genre_Free to Play,genre_Massively Multiplayer
count,6998.000,6998.000,6998.0,6998.000,6998.000,6998.000,6998.000,6.998e+03,6998.000,6998.000,6998.000,6.998e+03,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6.998e+03,6998.000,6.998e+03,6998.000,6.998e+03,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000,6998.000
mean,3498.500,406504.949,0.0,0.821,0.265,0.162,0.377,1.362e+09,-0.662,33.148,28.058,2.019e+03,9.420,4.616,42.213,0.704,13.878,-0.413,-0.940,1.075,1.327,-0.193,2.681e+05,7440.700,1.867e+05,51.475,1.288e+04,6.754,2.589,6.761,6.411,1.871,4.487,0.416,0.525,0.197,0.161,0.144,0.263,0.190,0.129,0.044,0.053,0.009,0.022
std,2020.293,175538.127,0.0,0.571,0.611,0.561,0.485,3.088e+08,0.473,47.783,40.832,2.829e+04,10.702,6.923,32.646,1.074,30.127,0.492,0.238,1.381,3.183,0.395,1.688e+06,11864.643,1.551e+06,35.677,1.713e+05,15.670,7.775,16.034,10.739,6.055,19.029,0.493,0.499,0.397,0.367,0.351,0.440,0.393,0.335,0.206,0.225,0.097,0.148
min,0.000,10.000,0.0,-1.000,-1.000,-1.000,0.000,-1.000e+00,-1.000,-1.000,-1.000,-1.000e+00,-1.000,-1.000,-1.000,0.000,-1.000,-1.000,-1.000,-1.000,0.000,-1.000,-1.000e+00,-1.000,-1.000e+00,-1.000,-1.000e+00,-1.000,-1.000,-1.000,1.000,0.000,-1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,1749.250,300290.000,0.0,1.000,0.000,0.000,0.000,1.400e+09,-1.000,-1.000,-1.000,-1.000e+00,0.000,0.590,-1.000,0.000,-1.000,-1.000,-1.000,-1.000,0.000,0.000,6.640e+02,967.000,5.820e+02,14.395,0.000e+00,0.000,-1.000,-1.000,3.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,3498.500,416635.000,0.0,1.000,0.000,0.000,0.000,1.461e+09,-1.000,-1.000,-1.000,-1.000e+00,6.990,2.490,50.000,0.000,-1.000,0.000,-1.000,2.000,0.000,0.000,1.188e+04,3027.000,7.053e+03,59.990,6.230e+02,0.880,1.500,2.900,3.000,0.000,3.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
75%,5247.750,547492.500,0.0,1.000,1.000,1.000,1.000,1.489e+09,0.000,100.000,79.627,2.210e+02,14.990,6.490,75.000,1.000,-1.000,0.000,-1.000,2.000,0.000,0.000,1.161e+05,9494.500,6.642e+04,80.475,2.433e+03,6.150,3.500,6.700,5.000,1.000,5.000,1.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000
max,6997.000,678620.000,0.0,1.000,1.000,1.000,1.000,1.595e+09,0.000,100.000,98.720,2.041e+06,79.990,59.990,100.000,3.000,96.000,0.000,0.000,2.000,27.000,0.000,1.045e+08,235425.000,1.044e+08,201.660,9.164e+06,151.190,253.900,298.000,234.000,64.000,1165.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000


In [23]:
feature_columns = ['OS_windows', 'OS_mac',
       'OS_linux', 'SteamPlay', 'Release_Date', 'worstRating', 'bestRating',
       'ratingValue', 'Price', 'Lowest_Price', 'Max_Sale',
       'controller_support', 'metacritic_score', 'community_visible_stats',
       'workshop_visible', 'releasestate', 'languages_num',
       'community_hub_visible', 
       'owners_played_percent', 'players_2_weeks_percent',
       'median_total_playtime', 'average_total_playtime', 'Packages', 'DLCs',
       'Depots', 'genre_Action', 'genre_RPG', 'genre_Indie', 'genre_Strategy',
       'genre_Adventure', 'genre_Simulation', 'genre_Casual',
       'genre_Early Access', 'genre_Racing', 'genre_Free to Play',
       'genre_Sports', 'genre_Massively Multiplayer']

"""removed columns, due to 'cheating' correlation with owners
'reviewCount', 'owners', 'owners_unc', 'players_total', 'players_2_weeks',

"""
target = 'owners'

y = df.loc[:,target]
X = df.loc[:,feature_columns]

#Split the data 70-30 train/test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [36]:
steps = [
('features', preprocessing.PolynomialFeatures(degree=2)),   
    #this looks at quadratic terms (3 would look at up to cubic in ALL combinations x1^2*x2, etc.)
('feature_selection', feature_selection.SelectFromModel(linear_model.Lasso(alpha = 25000000))),
    #Lasso model deletes the least useful features, only keeping the best, tuned by 'alpha'
('model',ensemble.GradientBoostingRegressor())]

regression_pipeline = pipeline.Pipeline(steps)
regression_pipeline.fit(X_train,y_train)
regression_pipeline.score(X_test,y_test)



/Users/dlicht/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


-1.249360288470069